In [27]:
import pandas as pd
from datetime import datetime
import time
from itertools import islice
import glob
import io

In [28]:
all_files = glob.glob('/bigdata/jelicicna/out_data/*.csv')
all_files.sort()

In [29]:
all_files

['/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-01.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-02.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-03.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-04.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-05.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-06.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-07.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-08.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-09.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-10.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-11.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-12.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-13.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-14.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-15.csv',
 '/bigdata/jelicicna/out_data/bikes_nextbike_2020-10-16.csv',
 '/bigda

In [30]:
li = []
for file in all_files:
    frame = pd.read_csv(file, sep= ';', index_col=None, header=0, low_memory=False)
    li.append(frame) 
df = pd.concat(li, axis=0, ignore_index=True, sort= False) #connecting all the dates

In [31]:
pd.set_option('display.max_columns()', None)

### Removing midnight breaks

In [32]:
namesToList = df['name'].tolist()
namesList = list(set(namesToList))

In [33]:
len(namesList)

1038

In [34]:
new_df = pd.DataFrame(columns= ['lat', 'lng', 'time_begin', 'time_end', 'name', 'address', 'bike', 'bike_racks', 'bikes', 
                                'booked_bikes', 'free_racks', 'free_special_racks', 'maintenance', 'number', 
                                'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type','uid',  'city',
                                'country_code', 'company', 'timezone', 'available_bikes'] ) #empty dataframe with the new column structure

In [35]:
for name in namesList: #repeat for each bike/bike name
    single_bike_records = df[df['name']== name] #separating dataframe that has all the columns as df but only for current name

    first_row_data = single_bike_records.iloc[0] #saving first row data
    latitude, longitude = first_row_data['lat'], first_row_data['lng']
    new_df = new_df.append({'lat' : latitude, 'lng' : longitude,
                            'time_begin' : first_row_data['time_begin'], 'time_end' : first_row_data['time_end'],
                            'name' : name, 'address' : first_row_data['address'], 'bike' : first_row_data['bike'], 
                            'bike_racks' : first_row_data['bike_racks'], 'bikes' : first_row_data['bikes'], 
                            'booked_bikes' : first_row_data['booked_bikes'], 'free_racks' : first_row_data['free_racks'], 
                            'free_special_racks' : first_row_data['free_special_racks'],
                            'maintenance' : first_row_data['maintenance'], 'number' : first_row_data['number'], 
                            'place_type' : first_row_data['place_type'], 'rack_locks' : first_row_data['rack_locks'], 
                            'special_racks' : first_row_data['special_racks'], 'spot' : first_row_data['spot'], 
                            'terminal_type' : first_row_data['terminal_type'],'uid' : first_row_data['uid'], 
                            'city' : first_row_data['city'], 
                            'country_code' : first_row_data['country_code'], 'company' : first_row_data['company'], 
                            'timezone' : first_row_data['timezone'],
                            'available_bikes' : first_row_data['available_bikes']}, ignore_index=True) 

    for index, row in islice(single_bike_records.iterrows(), 1, None): 
        if latitude == row['lat'] and longitude == row['lng']:
            new_df.iloc[-1, new_df.columns.get_loc('time_end')] = row['time_end'] 

        else: 
            new_df = new_df.append({'uid' : row['uid'] , 'lat' : row['lat'], 'lng' : row['lng'],
                            'time_begin' : row['time_begin'], 'time_end' : row['time_end'], 'name' : row['name'],
                            'address' : row['address'], 'bike' : row['bike'], 
                            'bike_racks' : row['bike_racks'], 'bikes' : row['bikes'], 
                            'booked_bikes' : row['booked_bikes'], 'free_racks' : row['free_racks'], 
                            'free_special_racks' : row['free_special_racks'], 
                            'maintenance' : row['maintenance'], 'number' : row['number'], 
                            'place_type' : row['place_type'], 'rack_locks' : row['rack_locks'], 
                            'special_racks' : row['special_racks'], 'spot' : row['spot'], 
                            'terminal_type' : row['terminal_type'], 'city' : row['city'], 
                            'country_code' : row['country_code'], 'company' : row['company'], 
                            'timezone' : row['timezone'],
                            'available_bikes' : row['available_bikes']}, ignore_index=True)
            #saving the new row if there is a change in location
            latitude, longitude = row['lat'], row['lng'] #updating to new location, continue searching for changes

In [36]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99846 entries, 0 to 99845
Data columns (total 25 columns):
lat                   99846 non-null float64
lng                   99846 non-null float64
time_begin            99846 non-null object
time_end              99846 non-null object
name                  99846 non-null object
address               621 non-null object
bike                  99846 non-null object
bike_racks            99846 non-null object
bikes                 99846 non-null object
booked_bikes          99846 non-null object
free_racks            99846 non-null object
free_special_racks    99846 non-null object
maintenance           99846 non-null object
number                99846 non-null object
place_type            99846 non-null object
rack_locks            99846 non-null object
special_racks         99846 non-null object
spot                  99846 non-null object
terminal_type         0 non-null float64
uid                   99846 non-null object
city          

In [37]:
namesToList2 = new_df['name'].tolist()
namesList2 = list(set(namesToList2))

In [38]:
len(namesList2)

1038

In [39]:
new_df.to_csv('/bigdata/jelicicna/output_datasets/cleaned_data_nextbike_2020-10.csv', sep=';', index=False)